<a href="https://colab.research.google.com/github/dxda6216/q10/blob/main/circadian_period_q10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a simple script to calculate Q10 values for circadian period
### length by using SciPy Optimize non-linear least squares fit on Colab.
### Copyright (c) 2022 by dxda6216 (dxda6216 AT gmail DOT com)
###
#@title Q10 calculator for circadian period
import numpy as np
import statistics
import pandas as pd
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt

#@markdown For usage, see [GitHub repository page](https://github.com/dxda6216/q10).

#@markdown Input data, then hit **Runtime** -> **Run all** (or press **Ctrl+F9**).

# Data description (plot title)
Data_description = "Mutant ABC" #@param {type:"string"}

# Temperature data
Temperatures = 25.1, 26.9, 27.3, 29.2, 30, 31.7, 32.2, 33.5, 34.1, 35.3, 35.8, 36.5, 37.1, 38.5, 38.9, 39.3, 40.3, 41.1 #@param {type:"raw"}
x = np.array(Temperatures)

# Period data
Periods =  33.6, 32.8, 30.4, 29.3, 28.0, 27.9, 26.1, 25.5, 24.9, 24.8, 24.5, 24.4, 24.1, 23.9, 23.6, 21.9, 21.5, 20.1 #@param {type:"raw"}
y = np.array(Periods)

median_x = statistics.median(x)
maximum_x = max(x)
minimum_x = min(x)
average_x = statistics.mean(x)

median_y = statistics.median(y)
maximum_y = max(y)
minimum_y = min(y)
average_y = statistics.mean(y)


# Pandas dataframe
tp = {'Temperature': x, 'Period': y}
df = pd.DataFrame(data=tp)
## Printing data
print("Input data")
print(df, "\n")
print(u'Minimum temperature =', '{:.3f}'.format(minimum_x), u'\u00B0C')
print(u'Maximum temperature =', '{:.3f}'.format(maximum_x), u'\u00B0C')
print(u'Average temperature =', '{:.3f}'.format(average_x), u'\u00B0C')
print(u'Median temperature  =', '{:.3f}'.format(median_x), u'\u00B0C\n')
print(u'Minimum period =', '{:.3f}'.format(minimum_y), u'h')
print(u'Maximum period =', '{:.3f}'.format(maximum_y), u'h')
print(u'Average period =', '{:.3f}'.format(average_y), u'h')
print(u'Median period  =', '{:.3f}'.format(median_y), u'h\n')

#@markdown Only data within a specific temperature range to be used for Q10 calculation? (Yes or No)

Set_temperature_range = "No" #@param ["Yes (set lowest and highest limits by sliders below)", "No"]
Range_low_limit = 20 # @param {type:"slider", min:0, max:60, step:0.1}
Range_high_limit = 50 # @param {type:"slider", min:0, max:60, step:0.1}

if Set_temperature_range == "No":
	dfa = df
else:
	if Range_low_limit < Range_high_limit:
		dfa = df[(df['Temperature'] >= Range_low_limit) & (df['Temperature'] <= Range_high_limit)]
	else:
		dfa = df

print("Data used for Q10 calculation")
xa = dfa['Temperature']
ya = dfa['Period']

median_xa = statistics.median(xa)
maximum_xa = max(xa)
minimum_xa = min(xa)
average_xa = statistics.mean(xa)

median_ya = statistics.median(ya)
maximum_ya = max(ya)
minimum_ya = min(ya)
average_ya = statistics.mean(ya)

print(dfa, "\n")
print(u'Minimum temperature =', '{:.3f}'.format(minimum_xa), u'\u00B0C')
print(u'Maximum temperature =', '{:.3f}'.format(maximum_xa), u'\u00B0C')
print(u'Average temperature =', '{:.3f}'.format(average_xa), u'\u00B0C')
print(u'Median temperature  =', '{:.3f}'.format(median_xa), u'\u00B0C\n')
print(u'Minimum period =', '{:.3f}'.format(minimum_ya), u'h')
print(u'Maximum period =', '{:.3f}'.format(maximum_ya), u'h')
print(u'Average period =', '{:.3f}'.format(average_ya), u'h')
print(u'Median period  =', '{:.3f}'.format(median_ya), u'h\n')


#@markdown Setting base temperature
# Base temperature
Select_base_temperature = "37°C" #@param ["-273.15\u00B0C (absolute zero)", "0\u00B0C", "4\u00B0C", "25\u00B0C", "30\u00B0C", "37\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 0 # @param {type:"slider", min:0, max:100, step:0.1}

if Select_base_temperature == "-273.15\u00B0C (absolute zero)":
	base_t = -273.15

if Select_base_temperature == "0\u00B0C":
	base_t = 0.000

if Select_base_temperature == "4\u00B0C":
	base_t = 4.000

if Select_base_temperature == "25\u00B0C":
	base_t = 25.000

if Select_base_temperature == "30\u00B0C":
	base_t = 30.000

if Select_base_temperature == "37\u00B0C":
	base_t = 37.000

if Select_base_temperature == "100\u00B0C":
	base_t = 100.000

if Select_base_temperature == "Minimum":
	base_t = minimum_xa

if Select_base_temperature == "Maximum":
	base_t = maximum_xa

if Select_base_temperature == "Average":
	base_t = average_xa

if Select_base_temperature == "Median":
	base_t = median_xa

if Select_base_temperature == "Set \"Base_temperature\" by slider below":
	base_t = Base_temperature

#@markdown Displaying tab-delimited data? (Yes or No)
# Displaying Tab-delimited data Yes or No
Display_tab_delimited_data = "No" #@param ["Yes", "No"]

# Defining an equation for curve fitting
# fitting parameters:
#     tau_bt : period at base median temperature (degree Celsius)
#     q10 : temperature coefficient (Q10)
def func(xa, tau_bt, q10):
	return tau_bt / ( q10 ** ( ( xa - base_t ) * 0.1 ) )

# Initial values for the fitting parameters
p0 = np.array([24.000, 1.000])

# Fitting the data to the defined equation
popt, pcov = curve_fit(func, xa, ya, p0)
residuals = ya - func(xa, *popt)
ss_residuals = np.sum(residuals**2)
ss_total = np.sum((ya-np.mean(ya))**2)
r_squared = 1 - ( ss_residuals / ss_total )

fig = plt.figure(figsize = (8,6))
fcxmin = int( min(x) - ( max(x) - min(x) ) * 0.333 )
fcxmax = int( max(x) + ( max(x) - min(x) ) * 0.333 ) + 1
fcx = np.linspace(fcxmin, fcxmax, 200)

plt.scatter(x, y, marker='o', s=35, color ='lightgreen', label ='Data')
plt.scatter(xa, ya, marker='x', s=15, color ='red', label ='Data points used for Q10 calculation')

fcy = func(fcx, popt[0], popt[1])
plt.plot(fcx, fcy, '--', color='blue', label ='Fit        Q10 = %5.3f' %popt[1])
# plt.title(Data_description)
plt.xlabel(u'Temperature (\u00B0C)')
plt.ylabel('Period (hours)')

if Display_tab_delimited_data == "Yes":
	print(u'Dataset')
	print (u'Temp (\u00B0C)\tPeriod (hours)')
	ycount = 0
	for xseq in x:
		print(str(xseq)+'\t'+str(y[ycount]))
		ycount += 1
	print(u'\nFitted Curve')
	print (u'Temp (\u00B0C)\tPeriod (hours)')
	fcycount = 0
	for fcxseq in fcx:
		print('{:.3f}'.format(fcxseq)+'\t'+'{:.3f}'.format(fcy[fcycount]))
		fcycount += 1
	print(u'\n')

# Printing the results
print(u'Eestimated period length at', '{:.3f}'.format(base_t), u'\u00B0C =', '{:.3f}'.format(popt[0]), u'\u00B1', '{:.3f}'.format(pcov[0,0]**0.5), 'hours')
print(u'Q10 (temperature coefficient) =', '{:.3f}'.format(popt[1]), u'\u00B1', '{:.3f}'.format(pcov[1,1]**0.5))
print(u'R\u00B2 =', '{:.6f}'.format(r_squared), u'\n')

# Adjustment of plot location
plt.subplots_adjust(top=0.9, bottom=0.0)

# Fig title, header, footer
fig.suptitle(Data_description, fontsize=12)
fst = 'Eest. tau at ' + '{:.2f}'.format(base_t) + ' \u00B0C = ' + '{:.3f}'.format(popt[0]) + ' \u00B1 ' + '{:.3f}'.format(pcov[0,0]**0.5) + ' hrs      Q10 = ' + '{:.3f}'.format(popt[1]) + ' \u00B1 ' + '{:.3f}'.format(pcov[1,1]**0.5)
fig.text(0.5, 0.92, fst, horizontalalignment="center", fontsize=9)
# fig.text(0.5, 0.015, "text", horizontalalignment="center", fontsize=9)

plt.legend()
plt.show()

### End of script